All data downloaded from [here](https://registry.opendata.aws/noaa-goes/)

Grid mappings that metpy expects to find in attributes somewhere must be one of [these](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#appendix-grid-mappings)

[Super useful GOES image description](https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm)

[Beginner's guide to GOES data](https://www.goes-r.gov/downloads/resources/documents/Beginners_Guide_to_GOES-R_Series_Data.pdf)

[GOES AOD Data description](https://www.star.nesdis.noaa.gov/goesr/documents/ATBDs/Baseline/ATBD_GOES-R_Aerosol_Optical_Depth_v4.2_Feb2018.pdf), table 3-22 on page 67 describes the output data
- [all GOES documents](https://www.goes-r.gov/resources/docs.html)

Also, some influence from the repositories from [here](https://github.com/xiaoheyu), which is the basis of [this paper](https://www.mdpi.com/2072-4292/13/23/4788)

[Short GOES-R aerosol training](http://cimss.ssec.wisc.edu/goes/goes-r/training/recordings/Aerosol/presentation_html5.html?lms=1)

[GOES-R reference(?) with a document to calculate SZA and LZA](https://www.ncei.noaa.gov/products/satellite/goes-r-series)

In [ ]:
import metpy
import datetime
import rioxarray
import os
import hvplot.xarray
import hvplot
import glob
import warnings

import geopandas as gpd
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.feature as cfeature
import holoviews as hv
import matplotlib.colors as colors
hv.extension('bokeh')

from pyproj import Proj
from tqdm.notebook import tqdm
from shapely.geometry import mapping
from dask import delayed
from dask.distributed import Client
from holoviews import opts
from pyorbital.orbital import get_observer_look

xr.set_options(keep_attrs=True)

In [ ]:
def get_projection(ds, variable):
    dat = ds.metpy.parse_cf(variable)
    return dat, dat.metpy.cartopy_crs
    
def change_xy_to_geospatial(ds, variable):
    dat = ds.metpy.parse_cf(variable)
    geos = dat.metpy.cartopy_crs
    
    x = dat.x
    y = dat.y
    
    sat_h = ds.goes_imager_projection.perspective_point_height
    ds.attrs["crs"] = geos
    ds = ds.assign_coords(x=ds.x * sat_h, y=ds.y * sat_h)
    
    ds.x.attrs = {'units': 'm',
     'axis': 'X',
     'long_name': 'X sweep in crs',
    }
    ds.y.attrs = {'units': 'm',
     'axis': 'Y',
     'long_name': 'Y sweep in crs',
    }

    return ds

In [ ]:
def listdir(path):
    for item in os.listdir(path):
        if item.startswith('.'):
            # ignore hidden files
            continue
        else:
            yield item

In [ ]:
regions = gpd.read_file('data/ne_50m_admin_1_states_provinces.zip')
states = regions[regions.admin == 'United States of America']
contiguous_us = states[(states.name != 'Alaska') & (states.name != 'Hawaii')]

# GOES Zenith Angle

In [ ]:
plate = ccrs.PlateCarree()

In [ ]:
with xr.open_dataset('/Users/kyle/Downloads/OR_ABI-L2-AODC-M6_G16_s20210411801068_e20210411803441_c20210411806227.nc') as ds:
    ds.load()


In [ ]:
a = ds.metpy.parse_cf('AOD')
proj = a.metpy.cartopy_crs

In [ ]:
satlat = ds.goes_imager_projection.latitude_of_projection_origin
satlon = ds.goes_imager_projection.longitude_of_projection_origin
sath = ds.goes_imager_projection.perspective_point_height
sattime = pd.to_datetime(ds.t.item()).to_pydatetime()

In [ ]:
xs, ys = np.meshgrid(np.arange(-180, 180), np.arange(-90, 90))

In [ ]:
az, el = get_observer_look(
    satlon, satlat, sath/1000,
    sattime,
    xs, ys, 0)

el = 90 - np.where(el < 0, np.nan, el)
# el = np.where(el <= 90, np.nan, el)

[compared to this](https://www.researchgate.net/figure/Limb-area-between-the-80-and-90-view-zenith-angle-isolines-of-the-GOES-16-dotted_fig1_353923891)

In [ ]:
from geographiclib.geodesic import Geodesic

line = Geodesic.WGS84.Line(
    lat1=ds.goes_imager_projection.latitude_of_projection_origin,
    lon1=ds.goes_imager_projection.longitude_of_projection_origin,
    azi1=-20
)

_xs, _ys = [], []

for level in np.arange(0, 90.1, 0.05):
    pos = line.ArcPosition(level)
    _xs.append(pos['lon2'])
    _ys.append(pos['lat2'])

az, angles = get_observer_look(
    satlon, satlat, sath/1000,
    sattime,
    _xs, _ys, 0)
angles = 90 - angles

it = iter(levels)
level = next(it)
idxs = []
for idx, a in enumerate(angles):
    if np.isclose(a, level, 0.05):
        idxs.append(idx)
        level = next(it, None)
        if level is None: break
            
label_x_locs = np.array(_xs)[idxs]
label_y_locs = np.array(_ys)[idxs]

transformed = proj.transform_points(x=label_x_locs, y=label_y_locs, src_crs=plate)
manual = list(zip(transformed[:,0], transformed[:,1]))

In [ ]:
# https://stackoverflow.com/a/18926541/5217293
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

def fmt(x):
    return f"{x:.0f}$^\circ$"

cmap = plt.get_cmap('inferno')
new_cmap = truncate_colormap(cmap, 0.2, 1.0)
fig, ax = plt.subplots(dpi=300, subplot_kw=dict(projection=proj))

levels = np.arange(10, 81, 10)
mappable = ax.contour(xs, ys, el, 
                      transform=plate, 
                      levels=levels, 
                      cmap=new_cmap, 
                      alpha=0.8,
                      linestyles='--')


ax.clabel(mappable, mappable.levels, 
          inline=True, 
          fmt=fmt, 
          fontsize=6, 
          colors='black',
          manual=manual
         )


fname = 'data/Natural_Earth_quick_start/50m_raster/NE1_50M_SR_W/NE1_50M_SR_W.tif'
img = plt.imread(fname)

ax.imshow(img, origin='upper', extent=[-180, 180, -90, 90], transform=ccrs.PlateCarree())

ax.scatter(satlon, satlat, transform=plate, s=10, color='red')
ax.add_feature(cfeature.STATES, lw=0.25)
ax.spines[:].set_visible(False)

save_path = '/Users/kyle/GoogleDrive/College/Graduate/Thesis/Figures and Data/tuned_models'
fig.savefig(f'{save_path}/GOES-Zenith-Angle.png', format='png')

In [ ]:
fig, ax = plt.subplots(dpi=300, subplot_kw=dict(projection=plate))

ax.coastlines()
ax.add_feature(cfeature.NaturalEarthFeature(
    'cultural', 'admin_0_countries', '50m',
    edgecolor='gray', facecolor='none'))

ax.scatter(satlon, satlat, s=10, color='red')
ax.imshow(img, origin='upper', extent=[-180, 180, -90, 90], transform=ccrs.PlateCarree())

ax.set_extent([-180, 180, -90, 90], plate)

ax.spines[:].set_visible(False)
gl = ax.gridlines(alpha=0.3, draw_labels=True)
gl.top_labels=False
gl.right_labels=False

# Exploring Data Quality

In [ ]:
data_directory = '/Volumes/Shores/GOES/AODC/2021/001'

In [ ]:
jan_1_files = sorted(glob.glob('/Volumes/Shores/GOES/AODC/2021/001/**/*.nc'))

In [ ]:
with xr.open_dataset(jan_1_files[10]) as temp:
    temp.load()

In [ ]:
def dqf_preprocess(ds):
    # flags
    # 0 high_quality_retrieval_qf 
    # 1 medium_quality_retrieval_qf 
    # 2 low_quality_retrieval_qf 
    # 3 no_retrieval_qf
    ds = ds.where(ds.DQF < 2)
    return ds

In [ ]:
ds = xr.open_mfdataset(
    jan_1_files,
    concat_dim="t", 
    parallel=True, 
    combine='nested',
    data_vars='different', 
#     preprocess=dqf_preprocess
)

In [ ]:
vals = ds.AOD.values.flatten()
vals = vals[np.isfinite(vals)] 
freq, edges = np.histogram(vals)

In [ ]:
high = ds.where(ds.DQF == 0).AOD.values.flatten()
high_freq, _ = np.histogram(high[np.isfinite(high)] , edges)

In [ ]:
med = ds.where(ds.DQF == 1).AOD.values.flatten()
med_freq, _ = np.histogram(med[np.isfinite(med)] , edges)

In [ ]:
low = ds.where(ds.DQF == 2).AOD.values.flatten()
low_freq, _ = np.histogram(low[np.isfinite(low)] , edges)

In [ ]:
no = ds.where(ds.DQF == 3).AOD.values.flatten()
no_freq, _ = np.histogram(no[np.isfinite(no)] , edges)

In [ ]:
fig, ax = plt.subplots(dpi=200)

offset = ((edges[1:] - edges[:-1]) / 5)
xs = ((edges[1:] + edges[:-1]) / 2) - offset
ax.grid(zorder=0)

alpha=0.7
width=0.09
ax.bar(xs, high_freq, 
       width=width, 
       label='High',
       alpha=alpha,
       zorder=2)

ax.bar(xs + 1*offset, med_freq, 
       width=width, 
       label='Medium', 
       alpha=alpha,
       zorder=2)

ax.bar(xs + 2*offset, low_freq, 
       width=width, 
       label='Low',
       alpha=alpha,
       zorder=2)

ax.legend(loc='upper right')

ax.set_yscale('log')
ax.set_xticks(np.round(edges, 2));

ax.tick_params(width=0, axis='y')
for spine in ['top', 'bottom', 'left', 'right']:
    ax.spines[spine].set_visible(False)

In [ ]:
fig, ax = plt.subplots(dpi=200)

counts, bins, bars = ds.AOD.where(ds.AOD > 0.45).plot(ax = ax, 
                edgecolor='tab:blue', 
                alpha=0.3, 
                rwidth=0.5,
               )

ax.set_ylim((0, ax.get_ylim()[1] * 1.2))
ax.set_title('AOD Frequencies, 24 hours, Jan 1, 2021')
ax.set_xticks(bins.round(2))
ax.grid(alpha=0.5)

for bar in bars:
    ax.text(bar.get_x(), bar.get_height()*1.02, f'{bar.get_height():.2e}',
            ha='left', rotation=45, fontsize='x-small')

ax.tick_params(width=0, which='both')
for spine in ['top', 'bottom', 'left', 'right']:
    ax.spines[spine].set_visible(False)

In [ ]:
lat_mean = ds.AOD.mean('x')
lon_mean = ds.AOD.mean('y')

In [ ]:
lat_plot = lat_mean.hvplot(x='y', y='t', title='Latitudinal mean, Jan 1, 2021').opts(axiswise=True)
lon_plot = lon_mean.hvplot(x='x', y='t', title='Longitudinal mean, Jan 1, 2021').opts(axiswise=True)

In [ ]:
vals = ds.AOD.values.flatten()

In [ ]:
plot = lat_plot + lon_plot

In [ ]:
hist, bin_edges = np.histogram(vals)

In [ ]:
warnings.filterwarnings('ignore')
plot.cols(1)

In [ ]:
hvplot.save(plot.cols(1), f'/Users/kyle/Downloads/GOES-AOD-spatial-averages.html')

# Averaging Per Hour

In [ ]:
data_directory = '/Volumes/Shores/GOES/AODC/'

In [ ]:
args = {}
bad_files = []

for year in ['2017', '2018', '2019', '2020']:
    year_files = {}
    year_path = os.path.join(data_directory, year)
    
    for day in listdir(year_path):
        day_path = os.path.join(year_path, day)

        for hour in listdir(day_path):
            hour_path = os.path.join(day_path, hour)
            hour_files = sorted(glob.glob(f'{hour_path}/*.nc'))

            multifile_input_path = sorted(glob.glob(f'{hour_path}/*.nc'))
            output_directory = os.path.join(f'/Volumes/Shores/GOES/AODC/averages/{year}/{day}')
            output_filename = f'{hour}.nc'

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)
            existing = year_files.get(day, [])
            existing.append({
                "multifile_input_path": multifile_input_path,
                "output_directory": output_directory,
                "output_filename": output_filename
            })
            year_files[day] = existing
    args[year] = year_files

In [ ]:
from pickle import dump, HIGHEST_PROTOCOL
with open('data/args.pcl', 'wb') as handle:
    dump(args, handle, protocol=HIGHEST_PROTOCOL)

In [ ]:
from pickle import load
with open('data/args.pcl', 'rb') as handle:
    args = load(handle)

In [ ]:
def dqf_preprocess(ds):
    # flags
    # 0 high_quality_retrieval_qf 
    # 1 medium_quality_retrieval_qf 
    # 2 low_quality_retrieval_qf 
    # 3 no_retrieval_qf
    global contiguous_us
    ds = ds[['AOD', 'DQF', 'goes_imager_projection']].where(ds.DQF < 2)
    
    new = change_xy_to_geospatial(ds, 'AOD')
    _, proj = get_projection(new, 'AOD')
    new.rio.write_crs(proj.to_string(), inplace=True)
    
    contiguous_us = contiguous_us.to_crs(new.rio.crs)
    clipped = new['AOD'].rio.clip(contiguous_us.geometry.apply(mapping))
    
    new['AOD'] = clipped
    ds.AOD.data = new['AOD'].data
    
    return ds

def take_hourly_average(multifile_input_path, output_directory, output_filename):
    output_path = os.path.join(output_directory, output_filename)
    
    if os.path.exists(output_path):
        return

    ds = xr.open_mfdataset(
        sorted(multifile_input_path), 
        concat_dim="t", 
        parallel=True, 
        combine='nested',
        data_vars='different', 
        preprocess = dqf_preprocess
    )
    
    ds['goes_imager_projection'] = ds.isel(t=0, x=0, y=0)['goes_imager_projection']
    
    avg = ds[['AOD', 'goes_imager_projection']].resample(t='1H').mean()
    del avg.attrs['crs']
    avg.to_netcdf(output_path)

In [ ]:
def run_args(args):
    errored_args = {}

    day_bar = tqdm(args.items())
    for day, params in day_bar:
        day_bar.set_description(f'{day}')
        for param in tqdm(params):
            try:
                take_hourly_average(**param)
            except Exception:
                existing = errored_args.get(day, [])
                existing.append(param)
                errored_args[day] = existing
    
    return errored_args

In [ ]:
# errors_2017 = run_args(args['2017'])

In [ ]:
# from pickle import dump, HIGHEST_PROTOCOL
# with open('data/2017-errors.pcl', 'wb') as handle:
#     dump(errors_2017, handle, protocol=HIGHEST_PROTOCOL)

In [ ]:
# errors_2018 = run_args(args['2018'])

In [ ]:
# from pickle import dump, HIGHEST_PROTOCOL
# with open('data/2018-errors.pcl', 'wb') as handle:
#     dump(errors_2018, handle, protocol=HIGHEST_PROTOCOL)

In [ ]:
# errors_2019 = run_args(args['2019'])

In [ ]:
# from pickle import dump, HIGHEST_PROTOCOL
# with open('data/2019-errors.pcl', 'wb') as handle:
#     dump(errors_2019, handle, protocol=HIGHEST_PROTOCOL)

In [ ]:
errors_2020 = run_args(args['2020'])

In [ ]:
from pickle import dump, HIGHEST_PROTOCOL
with open('data/2020-errors.pcl', 'wb') as handle:
    dump(errors_2020, handle, protocol=HIGHEST_PROTOCOL)

# Averaging Per Day

In [ ]:
data_directory = '/Volumes/Shores/GOES/AODC/averages'

In [ ]:
year_bar = tqdm(['2017', '2018', '2019', '2020'])

errors = {}

for year in year_bar:
    year_bar.set_description(year)
    daily_average_directory = os.path.join(data_directory, year, 'daily')
    year_directory = os.path.join(data_directory, year)
    
    os.makedirs(daily_average_directory, exist_ok=True)
    
    days = list(listdir(year_directory))
    day_bar = tqdm(days)
    for day in day_bar:
        try:
            day_bar.set_description(day)

            out = os.path.join(daily_average_directory, f'{day}.nc')

            if os.path.exists(out):
                continue

            day_path = os.path.join(year_directory, day)

            if 'daily' in day_path:
                continue

            hours = sorted([os.path.join(day_path, i) for i in listdir(day_path)])
            if len(hours) > 0:
                ds = xr.open_mfdataset(
                    hours,
                    concat_dim="t", 
                    parallel=True, 
                    combine='nested',
                    data_vars='different'
                )

                daily_average = ds.resample(t='1D').mean()
                daily_average['goes_imager_projection'] = daily_average.goes_imager_projection.isel(t=0)
                daily_average.to_netcdf(out)

                del ds
                del daily_average
        except:
            existing = errors.get(year, [])
            existing.append(day)
            errors[year] = existing

In [ ]:
ds.sel(t=pd.to_datetime(int(day)-1, unit='D', origin=year).strftime('%Y-%m-%d')).resample(t='1d').mean()